In [ ]:
import os, json,unstructured_client
from unstructured_client.models import operations, shared
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError
from pathlib import Path
from typing import List, Tuple
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()
client = unstructured_client.UnstructuredClient(
    api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
)
def get_elements_from_file(file_path:str,strategy:shared.Strategy=shared.Strategy.FAST):
    with open(file_path, "rb") as f:          # pass bytes, not a file handle
        file_bytes = f.read()
            # print first 10 bytes
    filename=os.path.basename(file_path)
    req = operations.PartitionRequest(
        partition_parameters=shared.PartitionParameters(
            strategy=strategy,
            files=shared.Files(
                content=file_bytes,
                file_name=filename,
                
            ),
            
            languages=["heb", "eng"],
        )
    )

    client = unstructured_client.UnstructuredClient(
        api_key_auth=os.getenv("UNSTRUCTURED_API_KEY")
    )

    try:  
        res = client.general.partition(request=req)
        elements = res.elements                          # list of Element objects
        
        return elements
    except SDKError as e:                                # friendlier error handling
        print(f"{e.status_code}: {e.message}")

def list_all_files(
    start: str | Path,
    exts: Tuple[str, ...] = (".pdf", ".doc", ".docx"),
) -> List[str]:
    """
    Return a list of full (absolute) paths for every *PDF, DOC, or DOCX* file
    inside *start* and all its sub-directories.

    Parameters
    ----------
    start : str | Path
        The root folder to walk. Tilde (~) is expanded and the path is
        resolved so the result is always absolute.
    exts : tuple[str, ...], optional
        File-name suffixes to include (case-insensitive). Default picks
        '.pdf', '.doc', and '.docx'.

    Raises
    ------
    FileNotFoundError
        If *start* does not exist or is not a directory.
    """
    root = Path(start).expanduser().resolve()

    if not root.is_dir():
        raise FileNotFoundError(f"{root} is not an existing directory")

    # rglob('*') walks recursively; filter by suffix (case-insensitive)
    return [
        str(p)
        for p in root.rglob("*")
        if p.is_file() and p.suffix.lower() in exts
    ]

    

In [ ]:
pathes=list_all_files("docs")

In [ ]:
for path in pathes:
    print(path)    

/Users/matansharon/python/AI/unsructered/docs/ArtPrompt- ASCII Art-based Jailbreak Attacks against Aligned LLMs.pdf
/Users/matansharon/python/AI/unsructered/docs/GaLore- Memory-Efficient LLM Training by Gradient Low-Rank Projection.pdf
/Users/matansharon/python/AI/unsructered/docs/2403.06634.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manifolds - Rev B_scanned for TF.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/Risk Management Plan - Bonded Manifolds.doc
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP - Bonded Manufolds - Rev A.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMF - bonded manifolds - Rev C.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMF- Bonded Manifolds -Rev G_scanned for TF.pdf
/Users/matansharon/python/AI/unsructered/docs/Products/Bonded Manifolds/obsolete/RMP -

In [219]:
unstructerd_files=[]
for i in range(3,7):
    if pathes[i].endswith(".pdf"):
        strategy=shared.Strategy.VLM
    elif pathes[i].endswith(".doc") or pathes[i].endswith(".docx"):
        strategy=shared.Strategy.FAST
    el=get_elements_from_file(pathes[i],strategy)
    unstructerd_files.append(el)


INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1

In [220]:
for file in unstructerd_files:
    print(file[0]['metadata']['filename'])
    


RMP - Bonded Manifolds - Rev B_scanned for TF.pdf
Risk Management Plan - Bonded Manifolds.doc
RMP - Bonded Manufolds - Rev A.pdf
RMF - bonded manifolds - Rev C.pdf


In [210]:
import pandas as pd
cols=['type','element_id','text','metadata','filename','page_number','text_as_html','language','filetype','partition_type']
metadata_cols=['filename','page_number','text_as_html','language','filetype','partition_type']
# df=pd.DataFrame(columns=cols)
df=pd.read_csv('unstructerd_df.csv')
df

,type,element_id,text,metadata,filename,page_number,text_as_html,language,filetype,partition_type
0,UncategorizedText,6b37173e9c5180e77bfe620773f2ef81,NaN,"{'category_depth': 0, 'page_number': 1, 'text_...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<div class=""Page"" data-page-number=""1"" id=""7d1...",NaN,application/pdf,NaN
1,Title,f04ab8375d52fc9874f380be7a6e2fd1,ArtPrompt: ASCII Art-based Jailbreak Attacks a...,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<h1 class=""Title"" id=""b2f3acf3a4a0427da98fe18d...",NaN,application/pdf,NaN
2,NarrativeText,4820ee097d1de1dce5cd53b612fa2c3e,⚠ WARNING: This paper contains model outputs t...,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<p class=""NarrativeText"" id=""7d3efa9742344716a...",NaN,application/pdf,NaN
3,UncategorizedText,46a00f49ac4707e94aef48456d972513,NaN,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<section class=""Section"" id=""dbde263fa0474785a...",NaN,application/pdf,NaN
4,NarrativeText,b9749709852d13efd2aca7020ae3c607,Fengqing Jiang*‡ Zhangchen Xu*‡ Luyao Ni...,"{'category_depth': 2, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<p class=""NarrativeText"" id=""8f1e0c539ff5483f8...",NaN,application/pdf,NaN
...,...,...,...,...,...,...,...,...,...,...
903,ListItem,c45eb708ec2a62b0e2f75b8c79aa6b28,Document Revision Control:,"{'category_depth': 0, 'filename': 'Risk Manage...",Risk Management Plan - Bonded Manifolds.doc,NaN,NaN,NaN,application/msword,NaN
904,Table,666d7a3208420e939c3f5648b6053d73,Revision History Tracking: Date: Revision 0- f...,{'emphasized_text_contents': ['Revision Histor...,Risk Management Plan - Bonded Manifolds.doc,NaN,<table><tr><td>Revision History Tracking:</td>...,NaN,application/msword,NaN
905,ListItem,beebf563bafcef299473597d0afd748b,Review and approval:,"{'category_depth': 0, 'filename': 'Risk Manage...",Risk Management Plan - Bonded Manifolds.doc,NaN,NaN,NaN,application/msword,NaN
906,Table,a2f1fc0cec9c9e2929faa08bbc51b8e0,Written by: Name Date Signature QA Eng. - - -,"{'emphasized_text_contents': ['Written by:', '...",Risk Management Plan - Bonded Manifolds.doc,NaN,<table><tr><td>Written by:</td><td>Name</td><t...,NaN,application/msword,NaN


In [195]:

def add_file_to_df(file: list, df: pd.DataFrame):
    
    for element in file:
        # Check if the element already exists in the DataFrame
        if df[df['element_id'] == str(element)].empty:
            # If it doesn't exist, create a new row
            pass
        else:
            # If it exists, skip to the next element
            continue
        # Create a dictionary for the row
        row_dict = {
            'type': element['type'],
            'element_id': str(element),
            'text': element['text'],
            'metadata': element['metadata'] 
        }
        
        # Extract each metadata column if it exists
        for meta_col in metadata_cols:
            if meta_col in element['metadata']:
                row_dict[meta_col] = element['metadata'][meta_col]
            else:
                row_dict[meta_col] = None  # Set to None if metadata field doesn't exist
                
        # Using concat instead of append (which is deprecated)
        df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)
    return df
# Iterate over the list of files and add each to the DataFrame





In [221]:
for file in unstructerd_files:
    df = add_file_to_df(file, df)
    # for el in file:
        # print(el)


/var/folders/hh/bn85jql94k9dkgccvt0vbxf40000gn/T/ipykernel_2982/4144970352.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)


In [222]:
df

,type,element_id,text,metadata,filename,page_number,text_as_html,language,filetype,partition_type
0,UncategorizedText,6b37173e9c5180e77bfe620773f2ef81,NaN,"{'category_depth': 0, 'page_number': 1, 'text_...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<div class=""Page"" data-page-number=""1"" id=""7d1...",NaN,application/pdf,NaN
1,Title,f04ab8375d52fc9874f380be7a6e2fd1,ArtPrompt: ASCII Art-based Jailbreak Attacks a...,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<h1 class=""Title"" id=""b2f3acf3a4a0427da98fe18d...",NaN,application/pdf,NaN
2,NarrativeText,4820ee097d1de1dce5cd53b612fa2c3e,⚠ WARNING: This paper contains model outputs t...,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<p class=""NarrativeText"" id=""7d3efa9742344716a...",NaN,application/pdf,NaN
3,UncategorizedText,46a00f49ac4707e94aef48456d972513,NaN,"{'category_depth': 1, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<section class=""Section"" id=""dbde263fa0474785a...",NaN,application/pdf,NaN
4,NarrativeText,b9749709852d13efd2aca7020ae3c607,Fengqing Jiang*‡ Zhangchen Xu*‡ Luyao Ni...,"{'category_depth': 2, 'page_number': 1, 'paren...",ArtPrompt- ASCII Art-based Jailbreak Attacks a...,1.0,"<p class=""NarrativeText"" id=""8f1e0c539ff5483f8...",NaN,application/pdf,NaN
...,...,...,...,...,...,...,...,...,...,...
1885,Table,"{'type': 'Table', 'element_id': '8f22aa04fd060...",6.2. Updated by: Signature Date Linda Ch. - QA...,"{'category_depth': 2, 'page_number': 1, 'paren...",RMF - bonded manifolds - Rev C.pdf,1.0,"<table class=""Table"" id=""72d00c29c92545ff85e37...",NaN,application/pdf,NaN
1886,Table,"{'type': 'Table', 'element_id': 'd56b6bcb40d98...",6.3. Reviewed & Approved by: Signature Date Mi...,"{'category_depth': 2, 'page_number': 1, 'paren...",RMF - bonded manifolds - Rev C.pdf,1.0,"<table class=""Table"" id=""5595c950a1374df4802d6...",NaN,application/pdf,NaN
1887,NarrativeText,"{'type': 'NarrativeText', 'element_id': '33430...",* end of document*,"{'category_depth': 2, 'page_number': 1, 'paren...",RMF - bonded manifolds - Rev C.pdf,1.0,"<p class=""NarrativeText"" id=""3f026b25c7754426a...",NaN,application/pdf,NaN
1888,UncategorizedText,"{'type': 'UncategorizedText', 'element_id': 'b...",,"{'category_depth': 2, 'page_number': 1, 'paren...",RMF - bonded manifolds - Rev C.pdf,1.0,"<footer class=""Footer"" id=""2ae7fd764e3d40c38a2...",NaN,application/pdf,NaN


In [223]:
df.to_csv('unstructured_df.csv', index=False)

In [224]:
len(df)

1890